In [126]:
import sys
sys.path.append("../../common/")
sys.path.append("../../")
import helper_fxns
reload(helper_fxns)
sys.path.append("..")
import architectures
reload(architectures)


<module 'architectures' from '../architectures.pyc'>

In [52]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *

In [39]:
def net_dense(args):
    conv_kwargs = dict(num_filters=args['k'], filter_size=3, pad=1, nonlinearity=rectify, W=lasagne.init.HeNormal())
    conv = InputLayer((None,16,96,96))
    conv = Conv2DLayer(conv, num_filters=128-(args["k"]*args["L"]), filter_size=5, stride=2, nonlinearity=rectify, name="c1")
    conv = helper_fxns.make_dense_block(conv, args, conv_kwargs)
    conv = Conv2DLayer(conv, num_filters=256-(args["k"]*args["L"]), filter_size=5, stride=2, nonlinearity=rectify, name="c2")
    conv = helper_fxns.make_dense_block(conv, args, conv_kwargs)
    conv = Conv2DLayer(conv, num_filters=512-(args["k"]*args["L"]), filter_size=5, stride=2, nonlinearity=rectify, name="c3")
    conv = helper_fxns.make_dense_block(conv, args, conv_kwargs)
    return conv

----

In [57]:
l_out, _ = architectures.climate_test_1()

<lasagne.layers.input.InputLayer object at 0x1319ca850> (None, 16, 96, 96)
<lasagne.layers.noise.GaussianNoiseLayer object at 0x1319ca990> (None, 16, 96, 96)
<lasagne.layers.conv.Conv2DLayer object at 0x12f25f490> (None, 128, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x12f25f5d0> (None, 256, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x12f25f810> (None, 512, 9, 9)
4148096


In [40]:
l_out = net({})
for layer in get_all_layers(l_out):
    print layer, layer.output_shape
print count_params(l_out)

<lasagne.layers.input.InputLayer object at 0x11947f5d0> (None, 16, 96, 96)
<lasagne.layers.conv.Conv2DLayer object at 0x11947fbd0> (None, 128, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1153ca210> (None, 256, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x1153ca610> (None, 512, 9, 9)
4148096


In [217]:
print 512-16*3
print 256-16*3
print 128-16*3

464
208
80


In [209]:
l_out = net_dense({"k":16,"L":3})
mains = [] # ok, tag the main conv layers
for layer in get_all_layers(l_out):
    print layer, layer.output_shape
    if layer.output_shape[1] in [512-16*3,256-16*3,128-16*3]:
        mains.append(layer)
print count_params(l_out)
print [elem.output_shape for elem in mains]

<lasagne.layers.input.InputLayer object at 0x1255ecf50> (None, 16, 96, 96)
<lasagne.layers.conv.Conv2DLayer object at 0x1255df810> (None, 80, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dfb10> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x1255dfc50> (None, 96, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dfd90> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x1255dfed0> (None, 112, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dffd0> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x102180850> (None, 128, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255df9d0> (None, 208, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x1255ca290> (None, 16, 21, 21)
<lasagne.layers.merge.ConcatLayer object at 0x1255ca3d0> (None, 224, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x1255ca510> (None, 16, 21, 21)
<lasagne.layers.merge.ConcatLayer object at 0x1255ca650> (None, 240, 21, 21)
<lasagne.layers

In [182]:
def make_inverse_dense_block(inp, layer, args):
    conc = inp

    #3 layers per comp unit and args['L'] units per block
    for lay in get_all_layers(layer)[::-1][:2*args['L']]:
        if isinstance(lay, ConcatLayer):
            conc = make_concat_inverse(conc,layer)
            
    return conc, lay.input_layer


def make_concat_inverse(inp, concat_layer):
    first_input_shape = concat_layer.input_shapes[0][concat_layer.axis]
    return SliceLayer(inp,indices=slice(0,first_input_shape), axis=concat_layer.axis)


----

In [234]:
#c2 = make_inverse_dense_block(l_out, l_out, {"L":3, "k":16})[1]
c2 = SliceLayer(l_out, indices=slice(0, 464), axis=1)
print c2, c2.output_shape
#print c2.input_layer.output_shape

<lasagne.layers.shape.SliceLayer object at 0x1255deb50> (None, 464, 9, 9)


In [231]:
c2a = TransposedConv2DLayer(c2, mains[-1].input_shape[1],
         mains[-1].filter_size, stride=mains[-1].stride, crop=mains[-1].pad,
         W=mains[-1].W, flip_filters=not mains[-1].flip_filters)
print c2a.output_shape

(None, 256, 21, 21)


In [232]:
#c1 = helper_fxns.make_inverse_dense_block(c2a, c2a, {"L":3, "k":16})[1]
c1 = SliceLayer(c2a, indices=slice(0,208), axis=1)
print c1, c1.output_shape

<lasagne.layers.shape.SliceLayer object at 0x1255de790> (None, 208, 21, 21)


In [276]:
c1a = TransposedConv2DLayer(c1, mains[-2].input_shape[1],
         mains[-2].filter_size, stride=mains[-2].stride, crop=mains[-2].pad,
         W=mains[-2].W, flip_filters=not mains[-2].flip_filters)
# hacky
c1a = PadLayer(c1a, width=[(1,0),(1,0)])
print c1a.output_shape

(None, 128, 46, 46)


In [277]:
c0 = SliceLayer(c1a, indices=slice(0,80), axis=1)
c0.output_shape

(None, 80, 46, 46)

In [279]:
c0a = TransposedConv2DLayer(c0, mains[-3].input_shape[1],
         mains[-3].filter_size, stride=mains[-3].stride, crop=mains[-3].pad,
         W=mains[-3].W, flip_filters=not mains[-3].flip_filters)
c0a = PadLayer(c0a, width=[(1,0),(1,0)])
print c0a.output_shape

(None, 16, 96, 96)


In [280]:
for layer in get_all_layers(c0a):
    print layer, layer.output_shape

<lasagne.layers.input.InputLayer object at 0x1255ecf50> (None, 16, 96, 96)
<lasagne.layers.conv.Conv2DLayer object at 0x1255df810> (None, 80, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dfb10> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x1255dfc50> (None, 96, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dfd90> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x1255dfed0> (None, 112, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255dffd0> (None, 16, 46, 46)
<lasagne.layers.merge.ConcatLayer object at 0x102180850> (None, 128, 46, 46)
<lasagne.layers.conv.Conv2DLayer object at 0x1255df9d0> (None, 208, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x1255ca290> (None, 16, 21, 21)
<lasagne.layers.merge.ConcatLayer object at 0x1255ca3d0> (None, 224, 21, 21)
<lasagne.layers.conv.Conv2DLayer object at 0x1255ca510> (None, 16, 21, 21)
<lasagne.layers.merge.ConcatLayer object at 0x1255ca650> (None, 240, 21, 21)
<lasagne.layers

In [91]:
SliceLayer(l_concat, indices=slice(20,30), axis=1).output_shape

(None, 10)

In [145]:
l_concat2.input_layers[0].input_layers[0].output_shape

(None, 10)